In [5]:
import os
import zipfile
import random
import tensorflow as tf
import shutil
from tensorflow.keras.optimizers import RMSprop
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from shutil import copyfile
import numpy as np
import matplotlib.pyplot as plt

In [9]:
# Local path variables
from datapath import data_path, zip_path

# Colab path variables
#zip_path = "sample_data/dogs-vs-cats.zip"
#data_path = "sample_data"

In [ ]:
# Extract the image directories from the zip files
zip_outer = zipfile.ZipFile(zip_path,'r')
zip_outer.extractall(data_path)
zip_outer.close()

zip_train = zipfile.ZipFile(zip_train_path)
zip_train.extractall(data_path)
zip_train.close()

zip_test = zipfile.ZipFile(zip_test_path)
zip_test.extractall(data_path)
zip_test.close()

del zip_outer, zip_train, zip_test

# delete zip files to reclaim space
os.remove(f"{data_path}/train.zip")
os.remove(f"{data_path}/test1.zip")
# os.remove(zip_path)

In [ ]:
def move_image(source,destination,image_name):
     shutil.move(os.path.join(source,image_name),
                 os.path.join(destination,image_name),
                 copy_function=copyfile)

In [ ]:
def sort_images(source_path,cat_path,dog_path,index_limit):
    for index,fname in enumerate(os.listdir(source_path)):
        if index < index_limit:
            move_image(source_path,cat_path,fname)
        else:
            move_image(source_path,dog_path,fname)
    
    shutil.rmtree(source_path)

In [12]:
# load dogs vs cats dataset, reshape and save to a new file

def load_images(image_dir, target_size):
    images = []
    labels = []
    for file in os.listdir(image_dir):
        image_label = 0
        if file.startswith('cat'):
            image_label = 1
        image = img_to_array(
            load_img(os.path.join(image_dir,file),target_size=target_size))
        images.append(image)
        labels.append(image_label)
    
    images = np.asarray(images)
    labels = np.asarray(labels)
    
    return images,labels

In [13]:
test_images, test_labels = load_images(os.path.join(data_path,'train'),(200,200))

In [16]:
test_images.shape, test_labels.shape

((25000, 200, 200, 3), (25000,))

In [20]:
#Keras model for image classification
model = tf.keras.models.Sequential([
    # Alternating convolution and pooling layers to process the images before analysis
    # The convulation layers accent features from the images,
    # The pooling layers reduce the amount of information to process
    tf.keras.layers.Conv2D(32,(3,3), activation='relu', input_shape=(200,200,3)),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(64,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128,(3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    # Flatten the images for analysis by the nn
    tf.keras.layers.Flatten(),
    # Neural network for image classification 256 neuron hidden layer, with a 1 nueron output layer
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation ='sigmoid')
])

model.compile(optimizer=RMSprop(learning_rate=0.0001),
              loss='binary_crossentropy',
              metrics=['acc'])

In [21]:
history = model.fit(test_images,test_labels, epochs = 2)

Epoch 1/2
782/782 [==============================] - 664s 847ms/step - loss: 0.8594 - acc: 0.6613
Epoch 2/2
782/782 [==============================] - 658s 842ms/step - loss: 0.4963 - acc: 0.7650
